# Project. Customer Analytics: Preparing Data for Modeling



![alt text](images/hr-image-small.png)

## Project Instructions

The Head Data Scientist at *Training Data Ltd.* has asked you to create a DataFrame called **ds_jobs** that stores the data in **customer_train.csv** much more efficiently. Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (**bool**).
- Columns containing integers only must be stored as 32-bit integers (**int32**).
- Columns containing floats must be stored as 16-bit floats (**float16**).
- Columns containing nominal categorical data must be stored as the **category** data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.  
The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.
If you call **.info()** or **.memory_usage()** methods on **ds_jobs** and **ds_jobs** after you've preprocessed it, you should notice a substantial decrease in memory usage.


A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called Training Data Ltd. to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to customer_train.csv, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                | Description                                                        |
|-----------------------|--------------------------------------------------------------------|
| student_id            | A unique ID for each student.                                      |
| city                  | A code for the city the student lives in.                          |
| city_development_index| A scaled development index for the city.                           |
| gender                | The student's gender.                                              |
| relevant_experience   | An indicator of the student's work relevant experience.            |
| enrolled_university   | The type of university course enrolled in (if any).                |
| education_level       | The student's education level.                                     |
| major_discipline      | The educational discipline of the student.                         |
| experience            | The student's total work experience (in years).                    |
| company_size          | The number of employees at the student's current employer.         |
| company_type          | The type of company employing the student.                         |
| last_new_job          | The number of years between the student's current and previous jobs.|
| training_hours        | The number of hours of training completed.                         |
| job_change            | An indicator of whether the student is looking for a new job (1) or not (0). |

In [ ]:

# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("data/customer_train.csv")    # Load the dataset

# View the dataset
# ds_jobs.head()
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [299]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

# 1. Exploratory data analysis

### 1.1. How to find the data types and contents of a column

In [300]:
def find_boolean_columns(df):
    boolean_columns = []
    for column in df.columns:
        if df[column].nunique() == 2:
            boolean_columns.append(column)
    return boolean_columns

# Find boolean columns in ds_jobs_transformed
boolean_columns = find_boolean_columns(ds_jobs_transformed)
print("Boolean columns:", boolean_columns)

Boolean columns: ['relevant_experience', 'job_change']


In [324]:
# EDA to help identify ordinal, nominal, and two-factor categories
for col in ds_jobs.select_dtypes("object").columns:
    print(ds_jobs_transformed[col].value_counts(), '\n')

city
city_103    805
city_16     211
city_114    167
city_21     157
city_160     72
           ... 
city_81       0
city_79       0
city_82       0
city_8        0
city_9        0
Name: count, Length: 123, dtype: int64 

gender
Male      1654
Female     145
Other       22
Name: count, dtype: int64 

relevant_experience
True    2201
Name: count, dtype: int64 

enrolled_university
no_enrollment       1960
Full time course     122
Part time course     103
Name: count, dtype: int64 

education_level
Graduate          1243
Masters            744
Phd                115
High School         75
Primary School       7
Name: count, dtype: int64 

major_discipline
STEM               1892
Humanities           73
Business Degree      48
Other                35
No Major             28
Arts                 21
Name: count, dtype: int64 

experience
>20    859
10     245
15     172
11     149
14     147
16     128
12     107
13     103
17      91
19      83
18      68
20      49
1        0
nan      0
<

# 2. Converting integers, floats, and unordered categories

Converting integers, floats, and unordered categories  
```py 'relevant_experience', 'job_change'```

In [301]:
def show_unique_values(df):
    unique_values = {column: df[column].nunique() for column in df.columns}
    return unique_values

# Show number of unique values in ds_jobs_transformed with the names of columns
unique_values = show_unique_values(ds_jobs_transformed)
print(unique_values)

{'student_id': 19158, 'city': 123, 'city_development_index': 93, 'gender': 3, 'relevant_experience': 2, 'enrolled_university': 3, 'education_level': 5, 'major_discipline': 6, 'experience': 22, 'company_size': 8, 'company_type': 6, 'last_new_job': 6, 'training_hours': 241, 'job_change': 2}


In [302]:
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype('bool')
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')

### candidates for   
"category": a small number of unique values with no natural ordering:  
gender, enrolled_university,  major_discipline, last_new_job

In [304]:
ds_jobs_transformed[[
    "gender", 
    "enrolled_university", 
    "major_discipline", 
    "last_new_job",
    "city",
    "company_type",
    "company_size"
]] = ds_jobs_transformed[[
    "gender", 
    "enrolled_university", 
    "major_discipline", 
    "last_new_job",
    "city",
    "company_type",
    "company_size"
]].astype('category')

### candidates for   
"ordered categories": containing categorical data with a natural ordering:  
education_level:  
education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308

In [305]:
ds_jobs_transformed['education_level'] = pd.Categorical(ds_jobs_transformed['education_level'], categories=[
        "Primary School", 
        "High School", 
        "Graduate", 
        "Masters", 
        "Phd"
        ], ordered=True)

### Columns containing integers only must be stored as 32-bit integers (int32)

In [306]:
ds_jobs_transformed[['student_id', 'training_hours']] = \
    ds_jobs_transformed[['student_id', 'training_hours']].astype('int32')

### Columns containing floats must be stored as 16-bit floats (float16)

In [307]:
ds_jobs_transformed['city_development_index'] = \
    ds_jobs_transformed['city_development_index'].astype('float16')

# 3. Converting ordered categories

### Creating an ordered list of work experience values

In [308]:
ds_jobs_transformed['experience'].value_counts()

experience
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: count, dtype: int64

You could use the .unique() and .sort() methods on the column, then convert to a list and rearrange to ensure <1 and >20 are in the correct position.

In [309]:
import re

def custom_sort(value):
    if value == '<1':
        return -1
    elif value == '>20':
        return float('inf')
    else:
        numeric_value = re.sub(r'\D', '', value)
        return int(numeric_value) if numeric_value else 0

unique_experience = ds_jobs_transformed['experience'].astype(str).unique()
sorted_experience = sorted(unique_experience, key=custom_sort)

print(sorted_experience)

['<1', 'nan', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']


Create a list of numbers 1-20, map the numbers to strings, and concatenate ["<1"] and [">20"] to the beginning and end, respectively.

In [310]:
sorted_experience2 = ['<1'] + [str(i) for i in range(1, 21)] + ['>20']
print(sorted_experience2)  

['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']


Manually

In [311]:
sorted_experience = ['nan', '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
sorted_enrolled_university = ['no_enrollment', 'Part time course', 'Full time course']
sorted_company_size = ['<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
sorted_last_new_job = ['never', '1', '2', '3', '4', '>4']

Converting columns into ordered categorical columns

In [312]:
from pandas.api.types import CategoricalDtype


# Convert 'experience' column to ordered categorical
#ds_jobs_transformed['experience'] = pd.Categorical(ds_jobs_transformed['experience'], categories=sorted_experience, ordered=True)
# experience
experience_dtype = CategoricalDtype(categories=sorted_experience, ordered=True)
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(experience_dtype)

# enrolled_university
enrolled_university_dtype = CategoricalDtype(categories=sorted_enrolled_university, ordered=True)   
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrolled_university_dtype)   

# company_size
company_size_dtype = CategoricalDtype(categories=sorted_company_size, ordered=True)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(company_size_dtype)

# last_new_job
last_new_job_dtype = CategoricalDtype(categories=sorted_last_new_job, ordered=True)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(last_new_job_dtype)



In [313]:
ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

# 4. Filtering on ordered categorical columns

In [314]:
ds_jobs_transformed['experience'].value_counts

<bound method IndexOpsMixin.value_counts of 0        >20
1         15
2          5
3         <1
4        >20
        ... 
19153     14
19154     14
19155    >20
19156     <1
19157      2
Name: experience, Length: 19158, dtype: category
Categories (23, object): ['nan' < '<1' < '1' < '2' ... '18' < '19' < '20' < '>20']>

In [315]:
ds_jobs_transformed['company_size'].value_counts()

company_size
50-99        3083
10000+       2019
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
100-500         0
10/49           0
Name: count, dtype: int64

Filtering DataFrames on ordered categorical columns

In [316]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & 
                                          (ds_jobs_transformed['company_size'].isin(['1000-4999', '5000-9999', '10000+']))]

#print('.info(): ', ds_jobs.info())
#print('.info(): ', ds_jobs_transformed.info())

print('**** memory_usage: ', ds_jobs.memory_usage())
print('** memory_usage: ', ds_jobs_transformed.memory_usage())

**** memory_usage:  Index                        132
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64
** memory_usage:  Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2941
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_ch

In [317]:
ds_jobs_transformed['job_change'].value_counts()

job_change
False    1908
True      293
Name: count, dtype: int64

In [318]:
ds_jobs_transformed['city'].value_counts()

city
city_103    805
city_16     211
city_114    167
city_21     157
city_160     72
           ... 
city_81       0
city_79       0
city_82       0
city_8        0
city_9        0
Name: count, Length: 123, dtype: int64

In [319]:
ds_jobs_transformed['company_type']

9              Pvt Ltd
12             Pvt Ltd
31             Pvt Ltd
34             Pvt Ltd
40       Public Sector
             ...      
19097          Pvt Ltd
19101          Pvt Ltd
19103          Pvt Ltd
19128          Pvt Ltd
19143          Pvt Ltd
Name: company_type, Length: 2201, dtype: category
Categories (6, object): ['Early Stage Startup', 'Funded Startup', 'NGO', 'Other', 'Public Sector', 'Pvt Ltd']

In [320]:
ds_jobs_transformed['enrolled_university'].value_counts()

enrolled_university
no_enrollment       1960
Full time course     122
Part time course     103
Name: count, dtype: int64

In [321]:
ds_jobs_transformed['company_size'].value_counts()

company_size
10000+       1095
1000-4999     796
5000-9999     310
<10             0
100-500         0
50-99           0
10/49           0
500-999         0
Name: count, dtype: int64

In [322]:
ds_jobs_transformed['last_new_job'].value_counts()

last_new_job
>4       828
1        608
2        347
4        172
3        155
never     74
Name: count, dtype: int64